In [5]:
from SocialNetwork.social_network_environment import *
from pprint import pprint
import numpy as np
import matplotlib.pyplot as plt
#from SocialNetwork.cascade import *
from Advertising.learners.Greedy_algorithm import *
from Advertising.learners.UCB_no_anything import *
from Advertising.learners.UCB_no_qualities import *

seed = 6789 
random.seed(seed)

exploration=4.2

deltas=[0.5,0.42,0.38,0.30,0.2,0.05]

social_network = social_network_environment()
social_network.init_from_json(json_path_network='Config/network.json', json_path_features='Config/features.json')

adversary_bids = []
for _ in range(0, 10): 
    adversary_i_bids = []
    for _ in range(0, 5):
        adversary_i_bids.append(random.randint(0, 4))
    adversary_bids.append(adversary_i_bids)

arms = arms_creation(seed = 30291, number_of_arms= 20)
n_rounds=10000
all_bids = adversary_bids

In [6]:
from Advertising.learners.UCB_Learner import *

vcg = VCG()

ucb_nq_f = UCB_Learner(arms = arms, hyperpar=exploration)
pulls=0
for t in range(0, n_rounds):
    if t%500 == 0:
        print("Still Going at round " + str(t))


    index=ucb_nq_f.pull_arm()
    best_arm = arms[index]


    all_bids.insert(0, best_arm)
    ad_allocation_list = setup(bids=all_bids, n_bids=5)
    best_allocation = vcg.all_best_allocations(ad_allocation_list, social_network)

    for j in range(0, len(best_allocation)):
        temp_allocation = []
        temp_allocation = best_allocation[j]
        for k in range(0, len(best_allocation[j])):
            if temp_allocation[k].ad_id == 1 and k != 0:
                temp_allocation[k].ad_id = None
                best_allocation[j] = temp_allocation

    # Computing payments
    payments = vcg.payments(ad_allocation_list, best_allocation, social_network)

    total_reward, active_nodes = activate_cascade(social_network=social_network, ad_allocation_list=best_allocation, slot_prominence=deltas,learner_id=1)

    payments_tot = calculate_total_payment(payments, social_network.categories, active_nodes)


    reward = (total_reward-payments_tot)
    

    ucb_nq_f.update(index, reward)

    all_bids.pop(0)
    reset_nodes(social_network)

print("Optimal arm, no qualities, first positions: "+str(arms[ucb_nq_f.return_best_arm()]))
print(ucb_nq_f.return_expected_values())


Still Going at round 0
Still Going at round 500
Still Going at round 1000
Still Going at round 1500
Still Going at round 2000
Still Going at round 2500
Still Going at round 3000
Still Going at round 3500
Still Going at round 4000
Still Going at round 4500
Still Going at round 5000
Still Going at round 5500
Still Going at round 6000
Still Going at round 6500
Still Going at round 7000
Still Going at round 7500
Still Going at round 8000
Still Going at round 8500
Still Going at round 9000
Still Going at round 9500
Optimal arm, no qualities, first positions: [3, 3, 4, 2, 3]
[-0.96304348  1.68525489  0.371       1.50128945  0.99764151  0.17964912
  0.30151515  1.5404342  -3.6225      0.06903846  1.42759904  1.4057162
  0.82133758 -1.43933333  0.          0.79066667  0.16775862 -0.297
  0.          0.81745098]


In [7]:
from Advertising.learners.UCB_Learner import *

vcg = VCG()

ucb_nq_a = UCB_Learner(arms = arms, hyperpar=exploration)

for t in range(0, n_rounds):
    if t%500 == 0:
        print("Still Going at round " + str(t))


    index=ucb_nq_a.pull_arm()
    best_arm = arms[index]


    all_bids.insert(0, best_arm)
    ad_allocation_list = setup(bids=all_bids, n_bids=5)
    best_allocation = vcg.all_best_allocations(ad_allocation_list, social_network)

    # Computing payments
    payments = vcg.payments(ad_allocation_list, best_allocation, social_network)

    total_reward, active_nodes = activate_cascade(social_network=social_network, ad_allocation_list=best_allocation, slot_prominence=deltas,learner_id=1)

    payments_tot = calculate_total_payment(payments, social_network.categories, active_nodes)


    reward = (total_reward-payments_tot)
    print(reward)


    ucb_nq_a.update(index, reward)

    all_bids.pop(0)
    reset_nodes(social_network)

print("Optimal arm, no qualities, all positions: "+str(arms[ucb_nq_a.return_best_arm()]))
print(ucb_nq_a.return_expected_values())

Still Going at round 0
4.780000000000002
3.6514285714285712
-2.7700000000000067
2.9100000000000072
7.688571428571429
4.240000000000002
-3.320000000000004
8.43
-1.9171428571428581
3.1926315789473723
-0.8666666666666689
-5.419999999999997
-1.9600000000000009
-5.1899999999999995
0
7.0
-6.429999999999996
-4.529999999999998
6.002380952380953
4.348095238095242
4.140000000000004
-1.3728571428571392
-4.04894736842105
-6.788471177944864
-1.7736842105263146
-4.251904761904763
-1.7036842105263137
2.980000000000003
-6.467994987468675
6.656315789473684
0
-9.110000000000007
6.729999999999993
-2.4199999999999973
12.014285714285712
-2.0900000000000007
7.959999999999999
5.714285714285716
-1.6900000000000008
5.361904761904764
8.07857142857143
-2.702380952380952
9.489999999999998
-0.3699999999999992
10.23
5.153333333333327
0.7633333333333372
-0.47666666666666613
7.966315789473683
-3.1700000000000035
5.7821052631578915
0.062105263157897994
11.420000000000002
-2.8499999999999996
2.3099999999999987
6.642631

In [8]:
from Advertising.learners.UCB_Learner_qualities import *

arms = arms_creation(seed = 30291, number_of_arms= 20)

vcg = VCG()

ucb_q_a = UCB_Learner_qualities(arms = arms, hyperpar=exploration, social_network = social_network)

for t in range(0, n_rounds):
    if t%500 == 0:
        print("Still Going at round " + str(t))

    index = ucb_q_a.pull_arm()
    best_arm = arms[index]  

    all_bids.insert(0, best_arm)
    ad_allocation_list = setup(bids=all_bids, n_bids=5)
    best_allocation = vcg.all_best_allocations(ad_allocation_list, social_network)  

    payments = vcg.payments(ad_allocation_list, best_allocation, social_network)
    active_nodes, click_rewards = active_nodes_click(social_network, best_allocation, deltas, learner_id=1)
    payments_tot = calculate_total_payment(payments, social_network.categories, active_nodes)

    reward = click_rewards - payments_tot

    ucb_q_a.update(index, reward, np.where(active_nodes==1)[0])

    all_bids.pop(0)
    reset_nodes(social_network)

print("Optimal arm, qualities, all positions: "+str(arms[ucb_q_a.return_best_arm()]))

Doing node number 0
Doing node number 1
Doing node number 2
Doing node number 3
Doing node number 4
Doing node number 5
Doing node number 6
Doing node number 7
Doing node number 8
Doing node number 9
Doing node number 10
Doing node number 11
Doing node number 12
Doing node number 13
Doing node number 14
Doing node number 15
Doing node number 16
Doing node number 17
Doing node number 18
Doing node number 19
Doing node number 20
Doing node number 21
Doing node number 22
Doing node number 23
Doing node number 24
Doing node number 25
Doing node number 26
Doing node number 27
Doing node number 28
Doing node number 29
Doing node number 30
Doing node number 31
Doing node number 32
Doing node number 33
Doing node number 34
Doing node number 35
Doing node number 36
Doing node number 37
Doing node number 38
Doing node number 39
Doing node number 40
Doing node number 41
Doing node number 42
Doing node number 43
Doing node number 44
Doing node number 45
Doing node number 46
Doing node number 47
Do

In [9]:
from Advertising.learners.UCB_Learner_qualities import *

arms = arms_creation(seed = 30291, number_of_arms= 20)

vcg = VCG()

ucb_q_f = UCB_Learner_qualities(arms = arms, hyperpar=exploration, social_network = social_network)

for t in range(0, n_rounds):
    if t%500 == 0:
        print("Still Going at round " + str(t))

    index = ucb_q_f.pull_arm()
    best_arm = arms[index]  

    all_bids.insert(0, best_arm)
    ad_allocation_list = setup(bids=all_bids, n_bids=5)
    best_allocation = vcg.all_best_allocations(ad_allocation_list, social_network)  

    for j in range(0, len(best_allocation)):
        temp_allocation = []
        temp_allocation = best_allocation[j]
        for k in range(0, len(best_allocation[j])):
            if temp_allocation[k].ad_id == 1 and k != 0:
                temp_allocation[k].ad_id = None
                best_allocation[j] = temp_allocation
    
    payments = vcg.payments(ad_allocation_list, best_allocation, social_network)
    active_nodes, click_rewards = active_nodes_click(social_network, best_allocation, deltas, learner_id=1)
    payments_tot = calculate_total_payment(payments, social_network.categories, active_nodes)

    reward = click_rewards - payments_tot

    ucb_q_f.update(index, reward, np.where(active_nodes==1)[0])

    all_bids.pop(0)
    reset_nodes(social_network)

print("Optimal arm, qualities, first positions: "+str(arms[ucb_q_f.return_best_arm()]))

Doing node number 0
Doing node number 1
Doing node number 2
Doing node number 3
Doing node number 4
Doing node number 5
Doing node number 6
Doing node number 7
Doing node number 8
Doing node number 9
Doing node number 10
Doing node number 11
Doing node number 12
Doing node number 13
Doing node number 14
Doing node number 15
Doing node number 16
Doing node number 17
Doing node number 18
Doing node number 19
Doing node number 20
Doing node number 21
Doing node number 22
Doing node number 23
Doing node number 24
Doing node number 25
Doing node number 26
Doing node number 27
Doing node number 28
Doing node number 29
Doing node number 30
Doing node number 31
Doing node number 32
Doing node number 33
Doing node number 34
Doing node number 35
Doing node number 36
Doing node number 37
Doing node number 38
Doing node number 39
Doing node number 40
Doing node number 41
Doing node number 42
Doing node number 43
Doing node number 44
Doing node number 45
Doing node number 46
Doing node number 47
Do